In [11]:
from Crypto.Cipher import Salsa20
from Crypto.Hash import HMAC, SHA256
from Crypto.Random import get_random_bytes

plainfile = open("file", 'rb')
secretSalsa20 = get_random_bytes(32)
secretSha256 = get_random_bytes(32)
with open("secret", 'wb') as passfile:
    passfile.write(secretSalsa20)
    passfile.write(secretSha256)
cipher = Salsa20.new(key=secretSalsa20)
msg = cipher.nonce + cipher.encrypt(plainfile.read())
h = HMAC.new(secretSha256, msg, digestmod=SHA256)
hex = h.hexdigest()
with open("Salsa20", 'wb') as salsafile:
    salsafile.write(msg)
with open("Sha256", 'w') as shafile:
    shafile.write(hex)

In [12]:
from Crypto.Hash import HMAC, SHA256
from Crypto.Cipher import Salsa20

passfile = open("secret", 'rb')
secretSalsa20 = passfile.read(32)
secretSha256 = passfile.read(32)
with open("Salsa20", 'rb') as salsafile:
    msg_nonce = salsafile.read(8)
    ciphertext = salsafile.read()
    msg = msg_nonce + ciphertext
    h = HMAC.new(secretSha256, msg, digestmod=SHA256)

try:
    with open("Sha256", 'r') as shafile:
        h.hexverify(shafile.read())
    cipher = Salsa20.new(key=secretSalsa20, nonce=msg_nonce)
    text = cipher.decrypt(ciphertext)
    with open('decrypted', 'wb') as decryptedfile:
        decryptedfile.write(text)
except ValueError:
    print("Message has been compromised")
passfile.close()